In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from mlflow import MlflowClient 
from mlflow.models import infer_signature
import mlflow
plt.style.use('fivethirtyeight')

In [2]:
ab_df = pd.read_csv('../../data/interim/absences_per_day.csv', sep=',')
#only keep columns index and count 
ab_df = ab_df[['index', 'count']]
#rename index to date
ab_df = ab_df.rename(columns={'index': 'date'})
ab_df['date'] = pd.to_datetime(ab_df['date'])
# drop data older than 2023-03-01
ab_df = ab_df[ab_df['date'] < '2023-03-01']

Test Train Split

In [3]:
split_date = '01-Jul-2022'
ab_train = ab_df.loc[ab_df['date'] <= split_date].copy()
ab_test = ab_df.loc[ab_df['date'] > split_date].copy()

In [4]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df['date']
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X

In [5]:
X_train, y_train = create_features(ab_train, label='count')
X_test, y_test = create_features(ab_test, label='count')

C:\Users\jakob\AppData\Local\Temp\ipykernel_19264\1007601217.py:12: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['weekofyear'] = df['date'].dt.weekofyear
C:\Users\jakob\AppData\Local\Temp\ipykernel_19264\1007601217.py:12: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['weekofyear'] = df['date'].dt.weekofyear


In [6]:
reg = xgb.XGBRegressor(n_estimators=1000)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        early_stopping_rounds=50,
       verbose=False) # Change verbose to True if you want to see it train
       
y_pred = reg.predict(X_test)


c:\Users\jakob\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [7]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [8]:
mlflow.set_tracking_uri('./mlruns')
mlflow.set_experiment('singlets')
experiment = mlflow.get_experiment_by_name('singlets')
client = MlflowClient()

run = client.create_run(experiment.experiment_id)

with mlflow.start_run(run_id=run.info.run_id) as run:

    (rmse, mae, r2) = eval_metrics(y_test, y_pred)
    
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    mlflow.xgboost.log_model(reg, 'model')